# Extract altitude

## Extract altitude from Swiss coordinates LV03

MNT25 / 200 is a topological model of Switzerland freely offered by the Swiss topological institute. It links every pair of coordinates from the old Swiss coordinate system (LV03) with an altitude based on the 1/25000 topological maps. This free version has a precision of 200m.

The model is quite heavy. You must download it [here](https://shop.swisstopo.admin.ch/fr/products/height_models/dhm25200) and place the file DHM200.xyz in the folder "data/MNT25".

In [1]:
#import modules
import pandas as pd
from tqdm import tqdm

In [2]:
#read csv file containing the coordinates E-W, N-S, altitude
alt = pd.read_csv("data/MNT25/DHM200.xyz", header=None, sep=" ", names=["E", "N", "altitude"])

In [3]:
#return a list of boolean based on the criterion : which are the observation with east coordinate equal to X
(alt["E"]==655600)

0          False
1          False
2          False
3           True
4          False
5          False
6          False
7          False
8          False
9          False
10         False
11         False
12         False
13         False
14         False
15         False
16         False
17         False
18         False
19         False
20         False
21         False
22         False
23         False
24         False
25         False
26         False
27         False
28         False
29         False
           ...  
1465586    False
1465587    False
1465588    False
1465589    False
1465590    False
1465591    False
1465592    False
1465593    False
1465594    False
1465595    False
1465596    False
1465597    False
1465598    False
1465599    False
1465600    False
1465601    False
1465602    False
1465603    False
1465604    False
1465605    False
1465606    False
1465607    False
1465608    False
1465609    False
1465610    False
1465611    False
1465612    False
1465613    Fal

In [4]:
#return a list of boolean based on the criterion : which are the observation with north coordinate equal to X
(alt["N"]==302000)

0           True
1           True
2           True
3           True
4           True
5           True
6           True
7           True
8           True
9           True
10          True
11          True
12          True
13          True
14          True
15          True
16          True
17          True
18          True
19          True
20          True
21          True
22          True
23          True
24          True
25          True
26          True
27          True
28          True
29          True
           ...  
1465586    False
1465587    False
1465588    False
1465589    False
1465590    False
1465591    False
1465592    False
1465593    False
1465594    False
1465595    False
1465596    False
1465597    False
1465598    False
1465599    False
1465600    False
1465601    False
1465602    False
1465603    False
1465604    False
1465605    False
1465606    False
1465607    False
1465608    False
1465609    False
1465610    False
1465611    False
1465612    False
1465613    Fal

In [5]:
#select the observation that satisfy the two preceding boolean conditions and extract the altitude
alt[(alt["E"]==655600)&(alt["N"]==302000)]["altitude"]

3    829.3
Name: altitude, dtype: float64

In our example, we see that the point with coordinates (E:655600, N:302000) is the observation with index 3 and the corresponding altitude is 829.3m.

However, not all possible pairs of coordinates are referenced in this free version. We want to build a function that return the altitude of the nearest point referenced.

In [6]:
#return the d nearest points
#however, it is not what we search because it doesn't return all points with E coordinates 655600 but a list of length n
#as a result, the point with the coordinate N of interest may not be in the list...
d=3
alt.loc[(alt['E']-655599).abs().argsort()[:d]]

,E,N,altitude
537727,655600.0,207200.0,816.9
361215,655600.0,229600.0,705.6
24199,655600.0,292800.0,882.1


In [7]:
#extract the value of the nearest E coordinate referenced
NNE = alt.loc[((alt['E']-655599).abs()).idxmin(), "E"]
NNE

655600.0

In [8]:
#extract the value of the nearest N coordinate referenced
NNN = alt.loc[((alt['N']-302005).abs()).idxmin(), "N"]
NNN

302000.0

In [9]:
#extract the altitude using the preceding code
alt[(alt["E"]==NNE)&(alt["N"]==NNN)]["altitude"]

3    829.3
Name: altitude, dtype: float64

In [10]:
#create a function for parameter E and N
def extract_altitude_LV03(topo_model, E, N):
    #extract the value of the nearest E coordinate referenced
    NNE = topo_model.loc[((topo_model['E']-E).abs()).idxmin(), "E"]
    
    #extract the value of the nearest N coordinate referenced
    NNN = topo_model.loc[((topo_model['N']-N).abs()).idxmin(), "N"]
    
    #extract the altitude using the preceding code
    altitude = topo_model[(topo_model["E"]==NNE)&(topo_model["N"]==NNN)]["altitude"]
    
    return altitude

In [11]:
#test the function on the coordinates used before
extract_altitude_LV03(alt, 655599, 302005)

3    829.3
Name: altitude, dtype: float64

In [12]:
#test the function on the coordinates of the Post St. Gallen (official altitude 668)
extract_altitude_LV03(alt, 745720, 254200)

209993    670.1
Name: altitude, dtype: float64

In [13]:
#test the function on the coordinates of the summit of the Bertolhütte (official altitude 3311)
extract_altitude_LV03(alt, 606981, 94963)

1411935    3238.7
Name: altitude, dtype: float64

In [14]:
#test the function on the coordinates of the summit of the Vanil Noir (official altitude 2389)
extract_altitude_LV03(alt, 577729, 153062)

1030061    2283.99
Name: altitude, dtype: float64

In [15]:
#test the function on the coordinates of the summit of the Ochse (official altitude 2188)
extract_altitude_LV03(alt, 598463, 171950)

853492    2109.1
Name: altitude, dtype: float64

We see that some an error is likely to occur for a summit because the difference of altitude with the nearest referenced point may by important. However, the function give a good approximation of the altitude.

In [16]:
alt.loc[1411935,:]

E           607000.0
N            95000.0
altitude      3238.7
Name: 1411935, dtype: float64

## Convert WGS84 coordinates to LV03

The coordinates available in metadata are based on WGS84 standard and is given in decimal degrees. In order to use the above defined function, we must first convert WGS84 coordinates to LV03. Formulas are provided by the [Swiss Topological Institute](https://www.swisstopo.admin.ch/en/maps-data-online/calculation-services.html) in a pdf file named `Approximate formulas for the transformation between Swiss projection coordinates and-WGS84.pdf`.

I created a function using those formulas.

In [17]:
#create a function to convert WGS84 coordinates to LV03
def convert_WGS84_to_LV03(latitude,longitude):
    f = latitude*3600 #transform decimal degrees to sexagesimal seconds
    l = longitude*3600 #transform decimal degrees to sexagesimal seconds
    f_p = (f-169028.66)/10000 #intermediate result
    l_p = (l-26782.5)/10000 #intermediate result
    E = 2600072.37 + 211455.93*l_p - 10938.51*l_p*f_p - 0.36*l_p*f_p**2 - 44.54*l_p**3 #get East coordinate LV95
    N = 1200147.07 + 308807.95*f_p + 3745.25*l_p**2 + 76.63*f_p**2 - 194.56*(l_p**2)*f_p + 119.79*f_p**3 #get North coordinate LV95
    e = round(E - 2000000) #get East coordinate LV03
    n = round(N - 1000000) #get North coordinate LV03
    
    return [e,n]

In [18]:
#try the function on the coordinates of the Post St. Gallen (47.42247,9.36997)
convert_WGS84_to_LV03(47.42247,9.36997)

[745720, 254200]

Trying the function on the coordinates of the Post St. Gallen (47.42247,9.36997), we note that we obtain exactly the LV03 coordinates we used in the first section. The function thus works and is very accurate. 

Then, I create a function that directly find the altitude from WGS84 coordinates in decimal degrees. As expected, with the same result as with extract_altitude_LV03().

In [19]:
def extract_altitude_WGS84(topo_model,latitude,longitude):
    #transform decimal degrees to sexagesimal seconds
    f = latitude*3600 
    l = longitude*3600
    
    #intermediate result
    f_p = (f-169028.66)/10000
    l_p = (l-26782.5)/10000
    
    #get LV95 coordinates
    E = 2600072.37 + 211455.93*l_p - 10938.51*l_p*f_p - 0.36*l_p*f_p**2 - 44.54*l_p**3
    N = 1200147.07 + 308807.95*f_p + 3745.25*l_p**2 + 76.63*f_p**2 - 194.56*(l_p**2)*f_p + 119.79*f_p**3
    
    #get LV03 coordinates
    e = round(E - 2000000) #get East coordinate LV03
    n = round(N - 1000000) #get North coordinate LV03
    
    #extract the value of the nearest E coordinate referenced
    NNE = topo_model.loc[((topo_model['E']-e).abs()).idxmin(), "E"]
    
    #extract the value of the nearest N coordinate referenced
    NNN = topo_model.loc[((topo_model['N']-n).abs()).idxmin(), "N"]
    
    #extract the altitude using the preceding code
    altitude = topo_model[(topo_model["E"]==NNE)&(topo_model["N"]==NNN)]["altitude"]
    
    return altitude

## Extract altitude variable from coordinates of observations

In [20]:
from PIL import Image
import matplotlib.pyplot as plt
import math

In [21]:
#load data
data = pd.read_csv("data/metadata/selection.csv")

In [22]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2528 entries, 0 to 2527
Data columns (total 10 columns):
Unnamed: 0                  2528 non-null int64
ML Catalog Number           2528 non-null int64
Scientific Name             2528 non-null object
Date                        2528 non-null object
Month                       2528 non-null int64
Locality                    2528 non-null object
Latitude                    2528 non-null float64
Longitude                   2528 non-null float64
Average Community Rating    2528 non-null float64
storage                     2528 non-null object
dtypes: float64(3), int64(3), object(4)
memory usage: 197.6+ KB


In [23]:
#load altitude model
alt = pd.read_csv("data/MNT25/DHM200.xyz", header=None, sep=" ", names=["E", "N", "altitude"])

In [24]:
#extract altitude of one observation
extract_altitude_WGS84(alt, data["Latitude"], data["Longitude"])

58    889.81
Name: altitude, dtype: float64

In [25]:
#get the WGS84 coordinates of this observation
data.loc[0,["Latitude","Longitude"]]

Latitude     47.4547
Longitude     9.4812
Name: 0, dtype: object

In [26]:
#create a of the same length as the data
altitude = pd.Series(range(len(data)))

#for each observation, extract the altitude from the WGS84 coordinates and store it in altitude
for i in tqdm(range(len(data))):
    altitude[i] = extract_altitude_WGS84(alt, data.loc[i,"Latitude"], data.loc[i,"Longitude"])

In [27]:
#define the Series altitute as a new variable of the metadata
data["altitude"] = altitude

## Transform WGS84 coordinates of observations in LV03 coordinates

In [28]:
#create Series of the length n observations
LV03_E = pd.Series(range(len(data)))
LV03_N = pd.Series(range(len(data)))

#for the n observations
for i in tqdm(range(len(data))):
    LV03 = convert_WGS84_to_LV03(data.loc[i,"Latitude"],data.loc[i,"Longitude"]) #get the swiss coordinates of the i-th
    LV03_E[i] = LV03[0] #store the E coordinate in LV03_E
    LV03_N[i] = LV03[1] #store the N coordinate in LV03_N

In [29]:
#add the Series as variables in the metadata dataframe
data["LV03_E"] = LV03_E
data["LV03_N"] = LV03_N

In [30]:
data.head()

,Unnamed: 0,ML Catalog Number,Scientific Name,Date,Month,Locality,Latitude,Longitude,Average Community Rating,storage,altitude,LV03_E,LV03_N
0,0,198885451,Aquila chrysaetos,2020-01-03,1,Grub/SG,47.4547,9.4812,0.0,data/images/aquila_chrysaetos/original/1988854...,715,754017,257995
1,1,198249561,Aquila chrysaetos,2016-09-10,9,Swiss National Park,46.6671,10.1934,0.0,data/images/aquila_chrysaetos/original/1982495...,1869,810775,172131
2,2,197139961,Aquila chrysaetos,2006-01-03,1,Pfynwald (zone générale),46.3035,7.6199,4.0,data/images/aquila_chrysaetos/original/1971399...,657,613965,128026
3,3,197006711,Aquila chrysaetos,2020-01-03,1,Grub/SG,47.4505,9.4869,0.0,data/images/aquila_chrysaetos/original/1970067...,870,754459,257539
4,6,192237111,Aquila chrysaetos,2019-12-08,12,Zermatt--Stadt (City of Zermatt),46.0215,7.7472,3.0,data/images/aquila_chrysaetos/original/1922371...,1609,623895,96708


In [31]:
#define new variable with path for the cropped images 
data["crop_storage"] = data.storage.str.replace("original","cropped")

In [32]:
#save metadata in a csv file
data.to_csv("data/metadata/selection_alt.csv",index=False)